## The Dockerfile
***
The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations.

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

Along the way, we clean up extra space. This makes the container smaller and faster to start.

Let's look at the Dockerfile for the example:

In [28]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
	 python3 \
	 python3-pip \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*


# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.

RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn && \
        pip install imbalanced-learn==0.4.3 && \

In [3]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
import sagemaker

## Building the container and registering with ECR
***
The following shell code shows how to build the container image using docker build and push the container image to ECR using docker push. This code is also available as the shell script container/build-and-push.sh, which you can run as build-and-push.sh churn-prediction to build the image sagemaker-churn-prediction.

This code looks for an ECR repository in the account you're using and the current default region (if you're using an Amazon SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [4]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-churn-prediction

cd container

chmod +x churn_test/train
chmod +x churn_test/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/9 : FROM ubuntu:18.04
18.04: Pulling from library/ubuntu
5667fdb72017: Pulling fs layer
d83811f270d5: Pulling fs layer
ee671aafb583: Pulling fs layer
7fc152dfb3a6: Pulling fs layer
7fc152dfb3a6: Waiting
d83811f270d5: Verifying Checksum
d83811f270d5: Download complete
ee671aafb583: Verifying Checksum
ee671aafb583: Download complete
7fc152dfb3a6: Verifying Checksum
7fc152dfb3a6: Download complete
5667fdb72017: Verifying Checksum
5667fdb72017: Download complete
5667fdb72017: Pull complete
d83811f270d5: Pull complete
ee671aafb583: Pull complete
7fc152dfb3a6: Pull complete
Digest: sha256:b88f8848e9a1a4e4558ba7cfc4acc5879e1d0e7ac06401409062ad2627e6fb58
Status: Downloaded newer image for ubuntu:18.04
 ---> 2ca708c1c9cc
Step 2/9 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Running in 1ca23073e6ad
Removing intermediate container 1ca23073e6ad
 ---> 6af504767015
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends 	 python3 	 python3-pip

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [5]:
# S3 prefix
prefix = 'Sage_churns'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [6]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Loading the required data to S3 bucket for training
***
When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic Iris dataset, which we have included.

We can use use the tools provided by the Amazon SageMaker Python SDK to upload the data to a default bucket.

In [7]:
#loading data to the data location
data_dir = 'data/churn.csv'
data_location = sess.upload_data(data_dir, key_prefix=prefix)

#loading training data for hyperparameter tuning
train_dir = 'train/train.csv'
train_location = sess.upload_data(train_dir, key_prefix=prefix)

#loading validation data for hyperparameter tuning
valid_dir = 'train/valid.csv'
valid_location = sess.upload_data(valid_dir, key_prefix=prefix)

#loading hyperparameters for hyperparameter tuning
hyperparameter_directory = 'container/local_test/test_dir/input/config/hyperparameters.json'
param_location = sess.upload_data(hyperparameter_directory, key_prefix=prefix)

In [8]:
hyperparameters = {"n_estimators":100,"max_depth":15,"max_features":55}

## Training the model
***
In order to use Amazon SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

 - The **container name**. This is constructed as in the shell commands above.
 - The **role**. As defined above.
 - The **instance count** which is the number of machines to use for training.
 - The **instance type** which is the type of machine to use for training.
 - The **output path** determines where the model artifact will be written.
 - The **session** is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [9]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-churn-prediction:latest'.format(account, region)

RF = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/Sage_churns".format(sess.default_bucket()),
                       hyperparameters = hyperparameters, 
                       sagemaker_session=sess)

RF.fit(data_location)

2019-10-09 10:57:07 Starting - Starting the training job...
2019-10-09 10:57:32 Starting - Launching requested ML instances......
2019-10-09 10:58:36 Starting - Preparing the instances for training......
2019-10-09 10:59:22 Downloading - Downloading input data
2019-10-09 10:59:22 Training - Downloading the training image..Starting the training.
(3299, 71) (3299,)
/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
Training completed using random forests classifier after upsampling.
recall_score = 0.9425287356321839

2019-10-09 10:59:54 Uploading - Uploading generated training model
2019-10-09 10:59:54 Completed - Training job completed
Training seconds: 44
Billable seconds: 44


## Hyperparameter tuning
***
In this section, we will use different hyperparameters and vary its value to determine the best fit and the best hyperparameters with respect to the mentioned metrics

In [10]:
RF.set_hyperparameters(num_round=100,
                       rate_drop=0.3,
                       tweedie_variance_power=1.4)

hyperparameter_ranges = {
                        'max_depth': IntegerParameter(1, 10, scaling_type='Linear'),
                        'n_estimators': IntegerParameter(1, 200, scaling_type='Linear'),
                        'max_features': IntegerParameter(1, 70, scaling_type='Linear')
                        }

In [11]:
objective_metric_name = 'recall_score'
objective_type = 'Maximize'
metric_definitions = [{'Name':'recall_score','Regex':'recall_score = (\S+)'}]

In [12]:
tuner = HyperparameterTuner(RF,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=5,
                            max_parallel_jobs=3,
                            objective_type = objective_type,
                            strategy='Random')

tuner.fit(data_location)

##  Launch Endpoint and Predictor Object
***
Here we launch the endpoint, deploy it and use the predictor object to predict the values of the testing set 

In [14]:
# Deployment using endpoint
from sagemaker.predictor import csv_serializer
predictor = RF.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer,endpoint_name='churn')

Using already existing model: sagemaker-churn-prediction-2019-10-09-10-57-07-196


--------------------------------------------------------------------------------------!

In [15]:
shape=pd.read_csv(valid_location, header=None)

In [16]:
print(predictor.predict(shape.values).decode('utf-8'))

0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0



# Invoke endpoint
***

In [18]:
endpoint_name = 'churn'
file_name = 'train/train.csv'

with open(file_name, 'r') as file:
    f = file.read()
    k = np.array(f)
    
client = boto3.client('sagemaker-runtime')
content_type = "text/csv"

response = client.invoke_endpoint(
    EndpointName = endpoint_name, 
    ContentType = content_type,
    Body = f)

x=response['Body'].read()
print(x.decode("utf-8"))

0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
1
0
0
1
0
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0


## Cleanup Endpoint
***
When you're done with the endpoint, you'll want to clean it up if not required.

In [15]:
sess.delete_endpoint(predictor.endpoint)

## Batch Transform Job
***
Now let's use the model built to run a batch inference job and verify it works.

In [19]:
transform_output_folder = "Sage_churns/output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = RF.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

Using already existing model: sagemaker-churn-prediction-2019-10-09-10-57-07-196


In [20]:
transformer.transform(valid_location, content_type='text/csv', split_type='Line')
transformer.wait()

....................Starting the inference server with 4 workers.
[2019-10-09 11:23:21 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2019-10-09 11:23:21 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2019-10-09 11:23:21 +0000] [10] [INFO] Using worker: gevent
[2019-10-09 11:23:21 +0000] [15] [INFO] Booting worker with pid: 15
[2019-10-09 11:23:21 +0000] [16] [INFO] Booting worker with pid: 16
[2019-10-09 11:23:21 +0000] [17] [INFO] Booting worker with pid: 17
[2019-10-09 11:23:21 +0000] [19] [INFO] Booting worker with pid: 19

169.254.255.130 - - [09/Oct/2019:11:23:43 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [09/Oct/2019:11:23:43 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Invoked with 335 records
169.254.255.130 - - [09/Oct/2019:11:23:43 +0000] "POST /invocations HTTP/1.1" 200 670 "-" "Go-http-client/1.1"


In [21]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/valid.csv.out".format(transform_output_folder), '/tmp/valid.csv.out')
with open('/tmp/valid.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

Transform results: 
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0



##  Package your resources as an Amazon SageMaker AlgorithmPackage
***
Now that we have verified that the algorithm code works for training, live inference and batch inference in the above sections, we have packaged it up as an Amazon SageMaker Algorithm.

## Region Limitation
***
Seller onboarding is limited to us-east-2 region (CMH) only. The client we are creating below will be hard-coded to talk to our us-east-2 endpoint only.

## Algorithm Definition
***
SageMaker Algorithm is comprised of 2 parts:
 - A training image
 - An inference image (optional)

The key requirement is that the training and inference images (if provided) remain compatible with each other. Specifically, the model artifacts generated by the code in training image should be readable and compatible with the code in inference image. You can reuse the same image to perform both training and inference or you can choose to separate them. This sample notebook has already created a single algorithm image that perform both training and inference. This image has also been pushed to your ECR registry at {{image}}. You need to provide the following details as part of this algorithm specification
## Training Specification
***
You specify details pertinent to your training algorithm in this section.
## Supported Hyper-parameters
***
This section captures the hyper-parameters your algorithm supports, their names, types, if they are required, default values, valid ranges etc. This serves both as documentation for buyers and is used by Amazon SageMaker to perform validations of buyer requests in the synchronous request path.

Please Note: While this section is optional, we strongly recommend you provide comprehensive information here to leverage our validations and serve as documentation. Additionally, without this being specified, customers cannot leverage your algorithm for Hyper-parameter tuning.

In [22]:
import boto3

smmp = boto3.client('sagemaker', region_name='us-east-1', endpoint_url="https://sagemaker.us-east-1.amazonaws.com")

In [23]:
from src.training_specification import TrainingSpecification
from src.training_channels import TrainingChannels
from src.metric_definitions import MetricDefinitions
from src.tuning_objectives import TuningObjectives
import json

training_specification = TrainingSpecification().get_training_specification_dict(
    ecr_image=image, 
    supports_gpu=True, 
    supported_channels=[
        TrainingChannels("training", description="Input channel that provides training data", supported_content_types=["text/csv"])], 
    supported_metrics=[MetricDefinitions("recall_score", "recall_score = (\S+)")],
    supported_tuning_job_objective_metrics=[TuningObjectives("Maximize", "recall_score")]
    )

print(json.dumps(training_specification, indent=2, sort_keys=True))

{
  "TrainingSpecification": {
    "MetricDefinitions": [
      {
        "Name": "recall_score",
        "Regex": "recall_score = (\\S+)"
      }
    ],
    "SupportedHyperParameters": [
      {
        "DefaultValue": "50",
        "IsRequired": false,
        "IsTunable": true,
        "Name": "max_features",
        "Range": {
          "IntegerParameterRangeSpecification": {
            "MaxValue": "70",
            "MinValue": "1"
          }
        },
        "Type": "Integer"
      },
      {
        "DefaultValue": "200",
        "IsRequired": false,
        "IsTunable": true,
        "Name": "n_estimators",
        "Range": {
          "IntegerParameterRangeSpecification": {
            "MaxValue": "500",
            "MinValue": "1"
          }
        },
        "Type": "Integer"
      },
      {
        "DefaultValue": "5",
        "IsRequired": false,
        "IsTunable": true,
        "Name": "max_depth",
        "Range": {
          "IntegerParameterRangeSpecification":

In [24]:
from src.inference_specification import InferenceSpecification
import json

inference_specification = InferenceSpecification().get_inference_specification_dict(
    ecr_image=image,
    supports_gpu=True,
    supported_content_types=["text/csv"],
    supported_mime_types=["text/csv"])

print(json.dumps(inference_specification, indent=4, sort_keys=True))

{
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": "130159455024.dkr.ecr.us-east-1.amazonaws.com/sagemaker-churn-prediction:latest"
            }
        ],
        "SupportedContentTypes": [
            "text/csv"
        ],
        "SupportedRealtimeInferenceInstanceTypes": [
            "ml.m4.xlarge",
            "ml.m4.2xlarge",
            "ml.m4.4xlarge",
            "ml.m4.10xlarge",
            "ml.m4.16xlarge",
            "ml.m5.large",
            "ml.m5.xlarge",
            "ml.m5.2xlarge",
            "ml.m5.4xlarge",
            "ml.m5.12xlarge",
            "ml.m5.24xlarge",
            "ml.c4.xlarge",
            "ml.c4.2xlarge",
            "ml.c4.4xlarge",
            "ml.c4.8xlarge",
            "ml.c5.xlarge",
            "ml.c5.2xlarge",
            "ml.c5.4xlarge",
            "ml.c5.9xlarge",
            "ml.c5.18xlarge",
            "ml.p2.xlarge",
            "ml.p2.8xlarge",
            "ml.p2.16xlarge",
         

In [25]:
from src.algorithm_validation_specification import AlgorithmValidationSpecification
import json

validation_specification = AlgorithmValidationSpecification().get_algo_validation_specification_dict(
    validation_role = role,
    training_channel_name = "training",
    training_input = train_location,
    batch_transform_input = valid_location,
    content_type = "text/csv",
    instance_type = "ml.c4.xlarge",
    output_s3_location = 's3://{}/{}'.format(sess.default_bucket(), prefix))

print(json.dumps(validation_specification, indent=4, sort_keys=True))

{
    "ValidationSpecification": {
        "ValidationProfiles": [
            {
                "ProfileName": "ValidationProfile1",
                "TrainingJobDefinition": {
                    "HyperParameters": {},
                    "InputDataConfig": [
                        {
                            "ChannelName": "training",
                            "CompressionType": "None",
                            "ContentType": "text/csv",
                            "DataSource": {
                                "S3DataSource": {
                                    "S3DataDistributionType": "FullyReplicated",
                                    "S3DataType": "S3Prefix",
                                    "S3Uri": "s3://sagemaker-us-east-1-130159455024/Sage_churns/train.csv"
                                }
                            },
                            "RecordWrapperType": "None"
                        }
                    ],
                    "OutputDataCon

In [26]:
import json
import time

algorithm_name = "Churn-prediction" + str(round(time.time()))

create_algorithm_input_dict = {
    "AlgorithmName" : algorithm_name,
    "AlgorithmDescription" : "Prediction of churn in telecom industry using Random forest classifier",
    "CertifyForMarketplace" : False
}
create_algorithm_input_dict.update(training_specification)
create_algorithm_input_dict.update(inference_specification)
create_algorithm_input_dict.update(validation_specification)

print(json.dumps(create_algorithm_input_dict, indent=4, sort_keys=True))

print ("Now creating an algorithm in SageMaker")

smmp.create_algorithm(**create_algorithm_input_dict)

{
    "AlgorithmDescription": "Prediction of churn in telecom industry using Random forest classifier",
    "AlgorithmName": "Churn-prediction1570620242",
    "CertifyForMarketplace": false,
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": "130159455024.dkr.ecr.us-east-1.amazonaws.com/sagemaker-churn-prediction:latest"
            }
        ],
        "SupportedContentTypes": [
            "text/csv"
        ],
        "SupportedRealtimeInferenceInstanceTypes": [
            "ml.m4.xlarge",
            "ml.m4.2xlarge",
            "ml.m4.4xlarge",
            "ml.m4.10xlarge",
            "ml.m4.16xlarge",
            "ml.m5.large",
            "ml.m5.xlarge",
            "ml.m5.2xlarge",
            "ml.m5.4xlarge",
            "ml.m5.12xlarge",
            "ml.m5.24xlarge",
            "ml.c4.xlarge",
            "ml.c4.2xlarge",
            "ml.c4.4xlarge",
            "ml.c4.8xlarge",
            "ml.c5.xlarge",
            "ml.c5.2xlar

{'AlgorithmArn': 'arn:aws:sagemaker:us-east-1:130159455024:algorithm/churn-prediction1570620242',
 'ResponseMetadata': {'RequestId': '84c17f3e-a463-46d4-ae73-fd0e1c51570e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '84c17f3e-a463-46d4-ae73-fd0e1c51570e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Wed, 09 Oct 2019 11:24:02 GMT'},
  'RetryAttempts': 0}}

## Status of Algorithm package
***
This part desribes the status of the package untill it reaches a terminal stage

In [27]:
import time
import json

while True:
    response = smmp.describe_algorithm(AlgorithmName=algorithm_name)
    status = response["AlgorithmStatus"]
    print (status)
    if (status == "Completed" or status == "Failed"):
        print (response["AlgorithmStatusDetails"])
        break
    time.sleep(5)

Pending
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed
{'ValidationStatuses': [{'Name': 'ValidationPro

## Debugging Creation Issues
***
Entity creation typically never fails in the synchronous path. However, the validation process can fail for many reasons. If the above Algorithm creation fails, you can investigate the cause for the failure by looking at the "AlgorithmStatusDetails" field in the Algorithm object or "ModelPackageStatusDetails" field in the ModelPackage object. You can also look for the Training Jobs / Transform Jobs created in your account as part of our validation and inspect their logs for more hints on what went wrong. If all else fails, please contact AWS Customer Support for assistance!

## List on AWS Marketplace
***
Next, please go back to the Amazon SageMaker console, click on "Algorithms" (or "Model Packages") and you'll find the entity you created above. If it was successfully created and validated, you should be able to select the entity and "Publish new ML Marketplace listing" from SageMaker console.
<img src="train/publish-to-marketplace-action.png"/>